In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

/Users/mariofilho/miniconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [7]:
data = pd.read_feather("train.f")
eras = sorted(data['era'].unique())
eras[:10], len(eras)

([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 120)

# Prequential Sliding

In [8]:
results_val = []

for era in range(2,121):
    print("Train era {} - Validation era {}".format(era - 1, era))
    

    train = data[data['era'] == era-1]
    val = data[data['era'] == era]
    
    X_train = train.filter(regex=r'feature')
    X_val = val.filter(regex=r'feature')

    y_train = train['target_kazutsugi']
    y_val = val['target_kazutsugi']
     

    mdl = LGBMRegressor(max_depth=5, num_leaves=2**5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, random_state=0)
    mdl.fit(X_train, y_train)
    
    predictions = pd.Series(mdl.predict(X_val))
    ranked_predictions = predictions.rank(pct=True, method="first")
    correlation = np.corrcoef(y_val, ranked_predictions)[0, 1]
    #print(correlation)
    
    results_val.append(correlation)
    print("Correlation {}".format(correlation))
    print()
   

Train era 1 - Validation era 2
Correlation 0.03969464753674341

Train era 2 - Validation era 3
Correlation 0.02058858218661952

Train era 3 - Validation era 4
Correlation -0.01475194934862246

Train era 4 - Validation era 5
Correlation -0.015094861354816407

Train era 5 - Validation era 6
Correlation -0.022366280240631524

Train era 6 - Validation era 7
Correlation -0.0069299496665320435

Train era 7 - Validation era 8
Correlation -0.13557612811445827

Train era 8 - Validation era 9
Correlation -0.03569007473005757

Train era 9 - Validation era 10
Correlation -0.006217411110746297

Train era 10 - Validation era 11
Correlation -0.015269347046276019

Train era 11 - Validation era 12
Correlation -0.054527107965336795

Train era 12 - Validation era 13
Correlation -0.0035282301351037787

Train era 13 - Validation era 14
Correlation 0.007168643949882627

Train era 14 - Validation era 15
Correlation -0.001766602331979027

Train era 15 - Validation era 16
Correlation -0.15088593749198895

Trai

In [9]:
np.median(results_val)

-0.0038875791965753063

In [10]:
np.min(results_val)

-0.22324906296933225

In [11]:
np.max(results_val)

0.08561008040114315

In [12]:
np.mean(results_val)

-0.005208882013881453

In [13]:
len(results_val)

119

# Prequential Sliding with Gap
- gap original - remover apenas era mais próxima do treino

In [14]:
results_val = []

for era in range(3,121):
    print("Train era {} - Validation era {}".format(era - 2, era))
    

    train = data[data['era'] == era-2]
    val = data[data['era'] == era]
    
    X_train = train.filter(regex=r'feature')
    X_val = val.filter(regex=r'feature')

    y_train = train['target_kazutsugi']
    y_val = val['target_kazutsugi']
     

    mdl = LGBMRegressor(max_depth=5, num_leaves=2**5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, random_state=0)
    mdl.fit(X_train, y_train)
    
    predictions = pd.Series(mdl.predict(X_val))
    ranked_predictions = predictions.rank(pct=True, method="first")
    correlation = np.corrcoef(y_val, ranked_predictions)[0, 1]
    #print(correlation)
    
    results_val.append(correlation)
    print("Correlation {}".format(correlation))
    print()
   

Train era 1 - Validation era 3
Correlation -0.0280388853784984

Train era 2 - Validation era 4
Correlation 0.00505827674230719

Train era 3 - Validation era 5
Correlation 0.011847097290604363

Train era 4 - Validation era 6
Correlation -0.0155339697134603

Train era 5 - Validation era 7
Correlation 0.012094982901663965

Train era 6 - Validation era 8
Correlation -0.005530469339811997

Train era 7 - Validation era 9
Correlation 0.05811944801487118

Train era 8 - Validation era 10
Correlation -0.0390438168415928

Train era 9 - Validation era 11
Correlation -0.05368467321084613

Train era 10 - Validation era 12
Correlation 0.0035846928722171904

Train era 11 - Validation era 13
Correlation -0.008572022264409344

Train era 12 - Validation era 14
Correlation -0.024926386348438072

Train era 13 - Validation era 15
Correlation 0.014483091614162091

Train era 14 - Validation era 16
Correlation 0.005108372502381544

Train era 15 - Validation era 17
Correlation 0.015170194963336443

Train era 16

In [15]:
np.median(results_val)

0.003983804652702899

In [16]:
np.min(results_val)

-0.08957529084629375

In [17]:
np.max(results_val)

0.08376725394067651

In [18]:
np.mean(results_val)

0.005207809207370971

In [19]:
len(results_val)

118